In [109]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [110]:
url = "https://www.oyorooms.com/search?checkin=15%2F04%2F2020&checkout=16%2F04%2F2020&city=Pune&country=india&coupon=&filters%5Bcity_id%5D=11&guests=1&location=Pune%2C+Maharashtra%2C+India&roomConfig=1&roomConfig%5B%5D=1&rooms=1&searchType=city"

In [112]:
response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})

In [113]:
response

<Response [200]>

In [114]:
data = response.text

In [115]:
soup = BeautifulSoup(data,"html.parser")

In [116]:
soup


<!DOCTYPE html>

<html dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<meta content="ie=edge; chrome=1" httpequiv="x-ua-compatible"/>
<meta content="lCkLu-7S0Eb5LOgDgmoWxSWzbopNfzKW01ZEe3ldsOc" name="google-site-verification"/>
<script type="text/javascript">
      	window.renderingSource='server'
        var host = window.location.hostname;
        if (!String.prototype.endsWith) {
	        String.prototype.endsWith = function(search, this_len) {
		        if (this_len === undefined || this_len > this.length) {
			        this_len = this.length;
		        }
	      	  return this.substring(this_len - search.length, this_len) === search;
	        };
        }
        if (! (host.endsWith("oyorooms.com") || host.endsWith("oyorooms.ms") || host.endsWith("oyohotels.co.jp") || host.endsWith("hotelesoyo.com") || host.endsWith("oyohotels.com") || host.endsWith("oyorooms.io")) && host.indexOf('localhost') < 0){
          window.location.href="https://www.oyorooms.com";
        }
        

In [117]:
titles = soup.find_all("div",{"class":"listing__contentWrapper"})

In [118]:
titles

[<div class="listing__contentWrapper"><div class="ListingContentHeader"><div class="ListingContentHeader__heading"><span><h1 class="ListingContentHeader__h1">183 OYOs in Pune,Maharashtra,India</h1></span><div class="ListingContentHeader__mapToggle"><span class="ListingContentHeader__mapToggle--label">Map View</span><label class="ToggleButton"><input class="ToggleButton__input" name="test" type="checkbox"/><span class="ToggleButton__sliderRound"></span></label></div><div><div class="dropdown"><span class="dropdown__label">Sort By</span><span class="dropdown__select" tabindex="-1">Popularity<ul class="dropdown__list"><li class="dropdown__item"><span>Popularity  </span></li><li class="dropdown__item"><span>Guest Ratings  </span></li><li class="dropdown__item"><span>Price Low to High  </span></li><li class="dropdown__item"><span>Price High to Low  </span></li></ul></span></div></div></div><div class="ListingContentHeader__divider"></div><div class="listingBanners"><div class="listingBanner

In [119]:
import re
from datetime import datetime
data = []
for i in titles:
    name = i.find_all("div",{"class":"oyo-cell--12-col oyo-cell--8-col-tablet oyo-cell--4-col-phone"})
    
    for j in name:
        hname = j.find("a",{"class":"c-nn640c u-width100"}).text
        address = j.find("span",{"itemprop":"streetAddress"}).text
        rating = j.find("div",{"class":"hotelRating"}).text
        if rating != "NEW":
            rt = rating[:3]
            a = rating[5:]
            rt_count = re.search('^\w+.\w+',a)
            rt_count = rt_count.group()
            x = rating.rfind(')')
            review = rating[x+2:]
        else:
            rt = "New"
            rt_count = "New"
            review = "New"
        amenity = j.find("div",{"class":"amenityWrapper"}).text        
        price = j.find("span",{"class":"listingPrice__finalPrice"}).text
        price = price[1:]
        oprice = j.find("span",{"class":"listingPrice__slashedPrice d-body-lg"}).text
        oprice = oprice[1:]
        disc = j.find("span",{"class":"listingPrice__percentage"}).text
        special = j.find("div",{"class":"c-1ubn3p0"})
        special = special.text if special else "No"
    
        #print(hname,address,rt,rt_count,review,special,amenity,price,oprice,disc,"\n----")
        data.append((datetime.now(),hname,address,rt,rt_count,review,special,amenity,price,oprice,disc))

In [120]:
data

[(datetime.datetime(2020, 9, 26, 17, 53, 22, 841025),
  'OYO 74535 Hotel Luxury Inn',
  'Ranjangaon Ganpati MIDC, Pune',
  '3.4',
  '17 Ratings',
  'Fair',
  'No',
  ' Parking Facility  Toiletries available  Reception  + 14 more',
  '849',
  '1415',
  '40% off'),
 (datetime.datetime(2020, 9, 26, 17, 53, 22, 854019),
  'OYO 46740 Paradise Inn',
  'Paud Road, Pune',
  '4.1',
  '36 Ratings',
  'Very Good',
  'WIZARD MEMBER',
  ' Reception  Intercom  Power backup  + 13 more',
  '1216',
  '2026',
  '39% off'),
 (datetime.datetime(2020, 9, 26, 17, 53, 22, 865012),
  'Capital O 44215 Hotel Majestic Executive',
  'Near Maiee Mangeshkar Hospital, Viola Society, Pune',
  '4.1',
  '23 Ratings',
  'Very Good',
  'WIZARD MEMBER',
  ' Center Table  Private Entrance  Parking Facility  + 18 more',
  '1546',
  '2577',
  '40% off'),
 (datetime.datetime(2020, 9, 26, 17, 53, 22, 873007),
  'OYO 71534 Hotel UK Executive',
  'Hanumanvadi, Pune',
  '3.6',
  '9 Ratings',
  'Good',
  'No',
  ' Parking Facility

In [121]:
df = pd.DataFrame(data,columns=['Datetime','Hotel Name','Address','Overall Rating','Rating Count',
                                'Review','Speciallity','Amenity','Price','Original Price','Discount Percentage'])
df.to_csv("E:\OyoScraper_Pune.csv",index=False)